In [60]:
import os
import glob
from pathlib import Path
import pandas
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# the path to the accuracy record
src = '/home/shq19004/Code/CONUSDisturbance/COLD/Calibration/'

path_list  = sorted(glob.glob(os.path.join(src, 'accuracy_record', 'accuracy_alldata_*.csv')))
path_list  = sorted(glob.glob(os.path.join(src, 'accuracy_record', 'accuracy_composite_*.csv')))
df_accuracy = []
for path_image in path_list:
    test_accuracy = pandas.read_csv(path_image)
    test_accuracy['Regrowth'] = int(Path(path_image).stem.split('_')[-1]) # append the regrowth value to the DataFrame at the end
    if len(df_accuracy) ==0:
        df_accuracy = test_accuracy
    else:
        df_accuracy = [df_accuracy, test_accuracy]
        df_accuracy = pandas.concat(df_accuracy)
df_accuracy.rename(columns={"F1Score": "F1 score", "Commission": "Commission error", "Omission": "Omission error"}, inplace=True) # for displaying the legend's title

df_accuracy_all = df_accuracy.copy()

In [71]:

from plotly.subplots import make_subplots
# plot the accuracy by updating the model
df_accuracy = df_accuracy_all.copy()
REGROWTH = 200
Conse = 6
Threshold = 0.95
df_accuracy = df_accuracy[(df_accuracy['Regrowth'] == REGROWTH) & (df_accuracy['Conse'] == Conse)& (df_accuracy['Threshold'] == Threshold)]

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
# Add traces
fig.add_trace(
    go.Scatter(x=df_accuracy['UpdateRate'], y=df_accuracy['F1 score'], name="F1 score"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_accuracy['UpdateRate'], y=df_accuracy['Commission error'], name="Commission error"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=df_accuracy['UpdateRate'], y=df_accuracy['Omission error'], name="Omission error"),
    secondary_y=True,
)

fig.update_layout( width=500, height=450)
# fig.update_layout(xaxis = dict(tickmode = 'array', tickvals = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]))
# fig.update_layout(yaxis_range=[0.05, 0.7])
# Adding symbol legend
fig.update_layout(
    legend=dict( title='',
        orientation='v',
        itemsizing='constant',
    )
)
fig.update_layout(template='ggplot2',font=dict(family='Arial', color='black'))
fig.show()

In [62]:
REGROWTHs = [200] # we first look at the accuracy of the regrowth == 200
MODEL_UPDATEs = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08]
for REGROWTH in REGROWTHs:
    for MODEL_UPDATE in MODEL_UPDATEs:
        df_accuracy = df_accuracy_all.copy()
        df_accuracy = df_accuracy[(df_accuracy['Regrowth'] == REGROWTH) & (df_accuracy['UpdateRate'] == MODEL_UPDATE)]

        print("Thershold of excluding regrowth: {}".format(REGROWTH))
        print("MODEL_UPDATE: {}".format(MODEL_UPDATE))
        print("Max F1 score: {}".format(df_accuracy['F1 score'].max()))
        CONS = [3, 4, 5, 6, 7, 8]
        CHANGE_PROB = [0.8, 0.85, 0.9, 0.95, 0.99] # , 0.999]
        df_accuracy = df_accuracy[(df_accuracy['Threshold'] > 0.8) & (df_accuracy['Threshold'] < 0.999)] #  [0.8, 0.85, 0.9, 0.95, 0.99] 
        # df_accuracy = df_accuracy[(df_accuracy['Threshold'] >= 0.95) & (df_accuracy['Threshold'] <= 0.95)] #  [0.8, 0.85, 0.9, 0.95, 0.99] 
        df_accuracy = df_accuracy[df_accuracy['Conse']> 3] #  [4, 5, 6, 7, 8]
        # Replace the values in the 'Conse' column of the DataFrame
        # df_accuracy['Conse'] = df_accuracy['Conse'].replace(3, 1)
        # df_accuracy['Conse'] = df_accuracy['Conse'].replace(4, 1)
        # df_accuracy['Conse'] = df_accuracy['Conse'].replace(5, 1.5)
        # df_accuracy['Conse'] = df_accuracy['Conse'].replace(6, 2)
        # df_accuracy['Conse'] = df_accuracy['Conse'].replace(7, 2.5)
        # df_accuracy['Conse'] = df_accuracy['Conse'].replace(8, 3)

        fig = px.scatter(df_accuracy, x="Commission error", y="Omission error", color="F1 score", size="Conse", # Threshold
                        color_continuous_scale = 'RdYlGn_r',
                        symbol = df_accuracy['Threshold'],
                        symbol_sequence= ['square-dot', 'circle-dot', 'diamond-dot',  'triangle-up-dot', 'star-dot'])
        fig.update_traces(marker=dict(line=dict(width=1, color='black')))
        fig.update_layout( width=500, height=450)
        fig.update_layout(xaxis = dict(tickmode = 'array', tickvals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]))
        fig.update_layout(yaxis = dict(tickmode = 'array', tickvals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]))
        fig.update_layout(xaxis_range=[0.05, 0.75])
        fig.update_layout(yaxis_range=[0.05, 0.75])
        # Adding symbol legend
        fig.update_layout(
            legend=dict( title='',
                orientation='v',
                x=0.745,
                y=0.98,
                itemsizing='constant',
            )
        )
        fig.update_layout(template='ggplot2',font=dict(family='Arial', color='black'))
        fig.show()

Thershold of excluding regrowth: 200
MODEL_UPDATE: 0
Max F1 score: nan


Thershold of excluding regrowth: 200
MODEL_UPDATE: 0.01
Max F1 score: 0.634119278779473


Thershold of excluding regrowth: 200
MODEL_UPDATE: 0.02
Max F1 score: 0.634833011316588


Thershold of excluding regrowth: 200
MODEL_UPDATE: 0.03
Max F1 score: 0.634990387256248


Thershold of excluding regrowth: 200
MODEL_UPDATE: 0.04
Max F1 score: 0.637211855104281


Thershold of excluding regrowth: 200
MODEL_UPDATE: 0.05
Max F1 score: 0.633097441480675


Thershold of excluding regrowth: 200
MODEL_UPDATE: 0.06
Max F1 score: 0.635675675675676


Thershold of excluding regrowth: 200
MODEL_UPDATE: 0.07
Max F1 score: 0.632146709816613


Thershold of excluding regrowth: 200
MODEL_UPDATE: 0.08
Max F1 score: 0.626641651031895


In [63]:
df_accuracy = df_accuracy_all.copy()
df_accuracy = df_accuracy[(df_accuracy['Conse'] == 6) & (df_accuracy['Threshold'] == 0.9)]

fig = px.scatter(df_accuracy, x="Commission error", y="Omission error", color="F1 score", size='Regrowth',
                 color_continuous_scale = 'RdYlGn_r',)
fig.update_traces(marker=dict(line=dict(width=1, color='black')))
fig.update_layout( width=500, height=450)
fig.update_layout(xaxis = dict(tickmode = 'array', tickvals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]))
fig.update_layout(yaxis = dict(tickmode = 'array', tickvals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]))
fig.update_layout(xaxis_range=[0.25, 0.45])
fig.update_layout(yaxis_range=[0.25, 0.45])
# Adding symbol legend
fig.update_layout(
    legend=dict( title='',
        orientation='v',
        x=0.745,
        y=0.98,
        itemsizing='constant',
    )
)

fig.update_layout(template='ggplot2',font=dict(family='Arial', color='black'))
fig.show()